# Step 1: Download Dataset

In [4]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Video_Games_5.json.gz
!gzip -d Video_Games_5.json.gz
!wget http://web.eecs.umich.edu/~mihalcea/affectivetext/data/WordNetAffectEmotionLists.tar.gz
!tar -xf WordNetAffectEmotionLists.tar.gz

--2021-05-09 00:02:45--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Video_Games_5.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 154050105 (147M) [application/octet-stream]
Saving to: ‘Video_Games_5.json.gz’

Video_Games_5.json. 100%[===================>] 146.91M  45.9MB/s    in 3.2s    

2021-05-09 00:02:48 (45.9 MB/s) - ‘Video_Games_5.json.gz’ saved [154050105/154050105]

--2021-05-09 00:02:53--  http://web.eecs.umich.edu/~mihalcea/affectivetext/data/WordNetAffectEmotionLists.tar.gz
Resolving web.eecs.umich.edu (web.eecs.umich.edu)... 141.212.113.214
Connecting to web.eecs.umich.edu (web.eecs.umich.edu)|141.212.113.214|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8287 (8.1K) [application/x-gzip]
Saving to: ‘WordNetAffectEmotionLists.tar.gz’

WordNetAffectEmotio 100%[=============

# **Step 2:Install the Scala Kernel**
If you get a "scala" kernel not recognized warning when loading up the notebook for the first time, start by running the two cells below. Once you are done **reload the page** to load the notebook in the installed Scala kernel.

In [5]:
!curl -Lo coursier https://git.io/coursier-cli && chmod +x coursier
!./coursier launch --fork almond:0.10.9 --scala 2.12 -- --install &> /dev/null 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   135  100   135    0     0    314      0 --:--:-- --:--:-- --:--:--   314
100 42577  100 42577    0     0  74565      0 --:--:-- --:--:-- --:--:-- 74565


In [1]:
println("(Testing)Hello, world!")

(Testing)Hello, world!


# **Step 3: Install dependentcies**

In [1]:
import $ivy.`org.slf4j:slf4j-api:2.0.0-alpha1`
import $ivy.`org.slf4j:slf4j-log4j12:2.0.0-alpha1`
import $ivy.`org.apache.spark::spark-core:3.1.1`
import $ivy.`org.apache.spark::spark-sql:3.1.1`
import $ivy.`org.apache.spark::spark-mllib:3.1.1`
// import $ivy.`com.nvidia::rapids-4-spark:0.4.1`
// import $ivy.`ai.rapids:cudf:0.18.1`
import $ivy.`log4j:log4j:1.2.17`
import $ivy.`org.jsoup:jsoup:1.13.1`
import $ivy.`org.plotly-scala::plotly-almond:0.8.2`
import coursierapi._
import sys.process._
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import $ivy.$                                 

import $ivy.$                                     

import $ivy.$                                   

import $ivy.$                                  

import $ivy.$                                    
// import $ivy.`com.nvidia::rapids-4-spark:0.4.1`
// import $ivy.`ai.rapids:cudf:0.18.1`

import $ivy.$                   

import $ivy.$                       

import $ivy.$                                      

import coursierapi._

import sys.process._

import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

import org.apache.log4j.{Level, Logger}


In [2]:
import org.apache.spark.sql._

implicit class RichDF(val df: DataFrame) {
  def showHTML(limit:Int = 20, truncate: Int = 20) = {
    import xml.Utility.escape
    val data = df.take(limit)
    val header = df.schema.fieldNames.toSeq
    val rows: Seq[Seq[String]] = data.map(_.toSeq.map(cell =>
        (cell match {
          case null => "null"
          case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
          case array: Array[_] => array.mkString("[", ", ", "]")
          case seq: Seq[_] => seq.mkString("[", ", ", "]")
          case _ => cell.toString
        }).take(truncate)
      ).map(p=> if(p.length >= truncate) p + "..." else p)
    )

    publish.html(s"""
      <table class="table dataframe" border="1">
        <tr>
        ${header.map(h => s"<th>${escape(h)}</th>").mkString}
        </tr>
        ${rows.map { row =>
          s"<tr>${row.map { c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
        }.mkString
        }
      </table>""")
  }
}

import org.apache.spark.sql._


defined class RichDF

# **Step 4: Describe Data**


In [3]:
import org.apache.spark.SparkConf
import org.apache.spark.SparkContext

val spark = NotebookSparkSession.builder()
    .appName("Product Satisfaction")
//     .config("spark.driver.resource.gpu.amount", "1")
    .config("spark.executor.memory","32g")
//     .config("spark.eventLog.enabled", "true")
    .config("spark.driver.cores", "4")
    .master("local[*]")
    .getOrCreate()
val sc = spark.sparkContext
val sqlContext= spark.sqlContext
// val conf = new SparkConf()
//             .set("spark.plugins", "com.nvidia.spark.SQLPlugin")
//             .set("spark.executor.resource.gpu.discoveryScript", "getGpusResources.sh")
//             .set("spark.rapids.sql.incompatibleOps.enabled", "true")
//             .set("spark.executor.resource.gpu.amount", "1")
//             .set("hadoop.home.dir","D:\\hadoop-3.2.2")
//             .set("spark.ui.enabled","true")
//             .setMaster("local[*]")
//             .setAppName("Product Satisfaction")


// val sc = new SparkContext(conf)
// val sqlContext= new SQLContext(sc)
// val spark=sqlContext.sparkSession

import spark.implicits._

//import org.jsoup.Jsoup
//kernel.publish.html(Jsoup.connect("http://8a1f1a0b9848:4040/").get().outerHtml())

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

import org.apache.spark.SparkConf

import org.apache.spark.SparkContext


spark: SparkSession = org.apache.spark.sql.SparkSession@191f477e
sc: SparkContext = org.apache.spark.SparkContext@78a8be67
sqlContext: SQLContext = org.apache.spark.sql.SQLContext@73ad284
import spark.implicits._

//import org.jsoup.Jsoup
//kernel.publish.html(Jsoup.connect("http://8a1f1a0b9848:4040/").get().outerHtml())

In [4]:
val amazonComment = spark.read.json("Video_Games_5.json")
amazonComment.printSchema()
amazonComment.showHTML()
amazonComment.count()

json at cmd3.sc:1

8 / 8

root
 |-- asin: string (nullable = true)
 |-- image: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- style: struct (nullable = true)
 |    |-- Color Name:: string (nullable = true)
 |    |-- Color:: string (nullable = true)
 |    |-- Configuration:: string (nullable = true)
 |    |-- Content:: string (nullable = true)
 |    |-- Denomination:: string (nullable = true)
 |    |-- Edition:: string (nullable = true)
 |    |-- Format:: string (nullable = true)
 |    |-- Item Package Quantity:: string (nullable = true)
 |    |-- Length:: string (nullable = true)
 |    |-- Offer Type:: string (nullable = true)
 |    |-- Package Quantity:: string (nullable = true)
 |    |-- Package Type:: string (nullable = true)
 |    |-- Pattern:: string (nullable = true)
 | 

take at cmd1.sc:6

1 / 1

...

count at cmd3.sc:4

8 / 8

count at cmd3.sc:4

1 / 1

amazonComment: DataFrame = [asin: string, image: array<string> ... 10 more fields]
res3_3: Long = 497577L

In [5]:
import java.io.File
val files = new File(".\\WordNetAffectEmotionLists").listFiles.map(_.getName).filter(_!="README")
val emotionLables = (files.map(f => sc.textFile("WordNetAffectEmotionLists/"+ f ).map(_.split(' ').drop(1)).flatMap(l=>l).map(f.dropRight(4) -> _)).fold(sc.emptyRDD[(String, String)])(_ ++ _)).toDF("emotion","word")
emotionLables.showHTML()

take at cmd1.sc:6

1 / 1

emotion,word
anger,wrath
anger,umbrage
anger,offense
anger,pique
anger,temper
anger,irritation
anger,lividity
anger,irascibility
anger,short_temper
anger,spleen


import java.io.File

files: Array[String] = Array(
  "anger.txt",
  "disgust.txt",
  "fear.txt",
  "joy.txt",
  "sadness.txt",
  "surprise.txt"
)
emotionLables: DataFrame = [emotion: string, word: string]

In [6]:
emotionLables.groupBy("emotion").count().show()

show at cmd5.sc:1

12 / 12

show at cmd5.sc:1

1 / 1

show at cmd5.sc:1

...

show at cmd5.sc:1

100 / 100

show at cmd5.sc:1

75 / 75

+--------+-----+
| emotion|count|
+--------+-----+
|     joy|  539|
|   anger|  318|
|    fear|  208|
|surprise|   90|
| sadness|  309|
| disgust|   72|
+--------+-----+



## **4.1 Transformate Data**

In [ ]:
import org.apache.spark.sql.functions.concat
val transformatedDF = amazonComment.select(($"overall" / 5).as("satisfaction") ,concat($"reviewText",$"summary").as("sentence"))
transformatedDF.show()
transformatedDF.count()

# **Step 5: Data Preparation**

## 5.1 Null Value Check

In [ ]:
val nullCheckedDF = transformatedDF.filter($"sentence".isNotNull)

### Word2Vec fecture transformation

In [8]:
//Word2Vec
import org.apache.spark.ml.feature.Tokenizer
import org.apache.spark.ml.feature.Word2Vec
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row

// Input data: Each row is a bag of words from a sentence or document.

val tokenizer = new Tokenizer().setInputCol("sentence").setOutputCol("words")
val wordsData = tokenizer.transform(nullCheckedDF.sample(0.5))
// Learn a mapping from words to Vectors.
val word2Vec = new Word2Vec()
  .setInputCol("words")
  .setOutputCol("result")
  .setVectorSize(128)
  .setMinCount(0)
val model = word2Vec.fit(wordsData)

val result = model.transform(wordsData)
result.show()

map at Word2Vec.scala:188

8 / 8

collect at Word2Vec.scala:191

8 / 8

repartition at Word2Vec.scala:347

...

1 / 1

show at cmd7.sc:19

1 / 1

+------------+--------------------+--------------------+--------------------+
|satisfaction|            sentence|               words|              result|
+------------+--------------------+--------------------+--------------------+
|         0.4|found the game a ...|[found, the, game...|[0.03550093228267...|
|         0.2|I'm an avid gamer...|[i'm, an, avid, g...|[-0.0672462206178...|
|         1.0|I bought this gam...|[i, bought, this,...|[-0.0696449068565...|
|         1.0|I have played the...|[i, have, played,...|[-0.0851883955403...|
|         0.8|I liked it and ha...|[i, liked, it, an...|[-0.0293926449680...|
|         0.2|I've bought and p...|[i've, bought, an...|[-0.0760261643830...|
|         1.0|Loved playing Dir...|[loved, playing, ...|[-0.0694960769342...|
|         1.0|lot of people don...|[lot, of, people,...|[0.00681758118598...|
|         0.8|I had Dirt 2 on X...|[i, had, dirt, 2,...|[-0.0753538312948...|
|         0.2|I'm sure I would ...|[i'm, sure, i, wo...|[-0.0853

import org.apache.spark.ml.feature.Tokenizer

import org.apache.spark.ml.feature.Word2Vec

import org.apache.spark.ml.linalg.Vector

import org.apache.spark.sql.Row

// Input data: Each row is a bag of words from a sentence or document.


tokenizer: Tokenizer = tok_f76cccd977e8
wordsData: DataFrame = [satisfaction: double, sentence: string ... 1 more field]
word2Vec: Word2Vec = w2v_9a09f7894df5
model: org.apache.spark.ml.feature.Word2VecModel = Word2VecModel: uid=w2v_9a09f7894df5, numWords=618756, vectorSize=128
result: DataFrame = [satisfaction: double, sentence: string ... 2 more fields]

### Seperate Training Dataset and Testing Dataset

In [9]:
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.sql.functions._
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.linalg.Vectors.fromML

val assembler = new VectorAssembler()
  .setInputCols(Array("result"))
  .setOutputCol("features")

val output = assembler.transform(result.select("satisfaction","result"))


val LabeledDF = output.map(p=>LabeledPoint(p.getDouble(0),fromML(p.getAs[Vector](1))))


val training,testing = LabeledDF.rdd.randomSplit(Array(0.7,0.3))
// val training_2,testing_2 = sc.union(testing).randomSplit(Array(0.7,0.3))

val sepTrainingDF = sc.union(training)
val sepTestingDF = sc.union(testing)

//StandardScaler
import org.apache.spark.mllib.feature.{StandardScaler, StandardScalerModel}

val trainScaler = new StandardScaler().fit(sepTrainingDF.map(p => p.features))
val trainScaleredData = sepTrainingDF.map(x=> (x.label, trainScaler.transform(x.features)))
val testScaler = new StandardScaler().fit(sepTestingDF.map(p => p.features))
val testScaleredData = sepTestingDF.map(x=> (x.label, testScaler.transform(x.features)))
import org.apache.spark.mllib.util.MLUtils
val trainingDF = MLUtils.convertVectorColumnsToML(trainScaleredData.toDF("label","features"))
val testingDF = MLUtils.convertVectorColumnsToML(testScaleredData.toDF("label","features"))
val labeledTrainingDF = trainingDF.map(p=>LabeledPoint(p.getDouble(0),fromML(p.getAs[Vector](1))))
val labeledTestingDF = testingDF.map(p=>LabeledPoint(p.getDouble(0),fromML(p.getAs[Vector](1))))

treeAggregate at Statistics.scala:58

16 / 16

treeAggregate at Statistics.scala:58

4 / 4

treeAggregate at Statistics.scala:58

16 / 16

treeAggregate at Statistics.scala:58

4 / 4

import org.apache.spark.ml.feature.VectorAssembler

import org.apache.spark.sql.functions._

import org.apache.spark.mllib.regression.LabeledPoint

import org.apache.spark.mllib.linalg.Vectors.fromML


assembler: VectorAssembler = VectorAssembler: uid=vecAssembler_8784a2155bc1, handleInvalid=error, numInputCols=1
output: DataFrame = [satisfaction: double, result: vector ... 1 more field]
LabeledDF: Dataset[LabeledPoint] = [label: double, features: vector]
training: Array[org.apache.spark.rdd.RDD[LabeledPoint]] = Array(
  MapPartitionsRDD[97] at randomSplit at cmd8.sc:16,
  MapPartitionsRDD[98] at randomSplit at cmd8.sc:16
)
testing: Array[org.apache.spark.rdd.RDD[LabeledPoint]] = Array(
  MapPartitionsRDD[99] at randomSplit at cmd8.sc:16,
  MapPartitionsRDD[100] at randomSplit at cmd8.sc:16
)
sepTrainingDF: org.apache.spark.rdd.RDD[LabeledPoint] = UnionRDD[101] at union at cmd8.sc:19
sepTestingDF: org.apache.spark.rdd.RDD[LabeledPoint] = UnionRDD[102] at union at cmd8.sc:20
import org.

# **Step 7: Algorithm Implementation**

In [10]:
val sample = testingDF.sample(0.1).take(10)


take at cmd9.sc:1

1 / 1

sample: Array[Row] = Array(
  [1.0,[-1.2093003636990556,1.3105324173584219,-1.7903171698969225,-0.4240970911356111,-1.6113658927683114,0.304135845014038,0.17452651743083075,-0.6661841969680904,0.6096387068793988,0.7370954769992213,0.10784863220971831,-0.565893864624726,-1.4252440197593912,2.139756095347843,-0.5120967393646094,-0.5197550532895954,-0.4063132050102405,2.4776764824624378,1.0163499719599371,-0.667743271569143,1.5558313252632157,-0.5125085069113372,0.2331457045649342,-0.3402290723255303,-0.9859623908151752,-0.25930134020244394,1.594465143563046,1.4802268119638036,-0.4961511626373065,0.209799494205105,-0.6693799690268875,-1.534986439109347,-0.6322034752714574,-1.3204175280127655,-3.4017513368735446,-1.1111321031256174,0.2948596486441022,-1.3180702583874155,2.023712690971348,0.04467184025059524,0.5741929758967045,-0.38097439808290906,-1.5110282252669018,0.45437041113626675,-0.20541634596846392,-1.4207908281062516,-0.27925197338299823,0.023204073471232763,0.6978050829858937,-1.

## Generalized Linear regression
---

### Train

In [11]:
import org.apache.spark.ml.regression.GeneralizedLinearRegression

val glr = new GeneralizedLinearRegression()
  .setFamily("gaussian")
  .setLink("identity")
  .setMaxIter(10)
  .setRegParam(0.3)

val glrModel = glr.fit(trainingDF)


head at MetadataUtils.scala:51

1 / 1

treeAggregate at WeightedLeastSquares.scala:107

16 / 16

treeAggregate at WeightedLeastSquares.scala:107

4 / 4

21/05/10 01:29:25 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
21/05/10 01:29:25 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


import org.apache.spark.ml.regression.GeneralizedLinearRegression


glr: GeneralizedLinearRegression = glm_2f5539e44188
glrModel: org.apache.spark.ml.regression.GeneralizedLinearRegressionModel = GeneralizedLinearRegressionModel: uid=glm_2f5539e44188, family=gaussian, link=identity, numFeatures=128

### Evaluate

In [12]:
// Print the coefficients and intercept for generalized linear regression model
println(s"Coefficients: ${glrModel.coefficients}")
println(s"Intercept: ${glrModel.intercept}")

// Summarize the model over the training set and print out some metrics
val summary = glrModel.summary
println(s"Coefficient Standard Errors: ${summary.coefficientStandardErrors.mkString(",")}")
println(s"T Values: ${summary.tValues.mkString(",")}")
println(s"P Values: ${summary.pValues.mkString(",")}")
println(s"Dispersion: ${summary.dispersion}")
println(s"Null Deviance: ${summary.nullDeviance}")
println(s"Residual Degree Of Freedom Null: ${summary.residualDegreeOfFreedomNull}")
println(s"Deviance: ${summary.deviance}")
println(s"Residual Degree Of Freedom: ${summary.residualDegreeOfFreedom}")
println(s"AIC: ${summary.aic}")
println("Deviance Residuals: ")
summary.residuals().show()


Coefficients: [-0.002204139737951326,0.002568639672345236,-0.0038227178212488296,0.0020170385134997528,-0.006041019491148093,8.193058284812231E-4,0.002081600305515067,-0.0013178618480638778,9.127207436916938E-4,-0.001955167042997399,4.110193471304427E-4,6.252108965872749E-4,-8.229225064353321E-4,0.0089346237271147,0.0028823468010909558,-0.005672864414997589,-0.0032428412352229416,0.005923725449345014,0.0020827663229662716,-0.002543767468305473,0.0012917957964711454,-0.0024135593794182993,0.009294443889570589,-0.0038858537417660742,-0.0016928973267258665,-0.0018260340223096288,0.0072147490059777,-0.005563310730250127,0.0031793066473255675,-0.0023953271914464243,-0.0029506651028735576,-0.004982605971689825,3.5131004076344174E-4,0.002610526212649966,-0.010323400753674768,0.0037539053443215397,-1.0237080615500883E-4,-0.004082662323027776,5.914670337447711E-4,0.002545570814834231,0.006573799157444667,6.261547797466354E-4,0.007882456221886132,-0.00216579103269052,0.0017342707897862204,0.0029

head at GeneralizedLinearRegression.scala:1248

16 / 16

head at GeneralizedLinearRegression.scala:1248

1 / 1

...

AIC: -63697.24964339997
Deviance Residuals: 


show at cmd11.sc:16

1 / 1

+--------------------+
|   devianceResiduals|
+--------------------+
|  -0.355252179988487|
| 0.18853737316299257|
|  -0.018736794559851|
| 0.16322586208941248|
| 0.20895995101913767|
|0.037949276228173345|
| 0.12674373438841924|
| 0.00893871548663705|
| -0.6002520323041298|
| -0.1717847496924716|
|0.006287406650118843|
|0.001331313955018...|
|  -0.561164438952112|
| 0.01665486044052078|
| 0.07025542202880675|
| 0.03497605073159504|
| 0.14702457524727564|
|-0.01811494574696182|
| 0.16603288166241625|
|-0.03072459411508...|
+--------------------+
only showing top 20 rows



summary: org.apache.spark.ml.regression.GeneralizedLinearRegressionTrainingSummary = Coefficients:
     Feature Estimate Std Error  T Value P Value
 (Intercept)   0.8062    0.0019 429.3200  0.0000
  features_0  -0.0022    0.0003  -6.4122  0.0000
  features_1   0.0026    0.0004   7.0761  0.0000
  features_2  -0.0038    0.0003 -11.5382  0.0000
  features_3   0.0020    0.0003   6.2073  0.0000
  features_4  -0.0060    0.0003 -18.3903  0.0000
  features_5   0.0008    0.0004   2.2764  0.0228
  features_6   0.0021    0.0004   5.8169  0.0000
  features_7  -0.0013    0.0004  -3.5561  0.0004
  features_8   0.0009    0.0004   2.6038  0.0092
  features_9  -0.0020    0.0004  -5.2830  0.0000
 features_10   0.0004    0.0003   1.2223  0.2216
 features_11   0.0006    0.0003   1.8349  0.0665
 features_12  -0.0008    0.0003  -2.4409  0.0147
 features_13   0.0089    0.0003  28.8421  0.0000
 features_14   0.0029    0.0004   7.9728  0.0000
 features_15  -0.0057    0.0004 -15.8286  0.0000
 features_16  -0.00

In [ ]:
import org.apache.spark.mllib.evaluation.RegressionMetrics
val predictionAndLabels = labeledTestingDF.rdd.map { case LabeledPoint(label, features) =>
  val prediction = glrModel.predict(features.asML)
  (prediction, label)
}
val metrics = new RegressionMetrics(predictionAndLabels)
// Squared error
println(s"MSE = ${metrics.meanSquaredError}")
println(s"RMSE = ${metrics.rootMeanSquaredError}")

// R-squared
println(s"R-squared = ${metrics.r2}")

// Mean absolute error
println(s"MAE = ${metrics.meanAbsoluteError}")

// Explained variance
println(s"Explained variance = ${metrics.explainedVariance}")


### Predict

In [14]:
sample.map(p=>(p.getDouble(0),glrModel.predict(p.getAs[Vector](1))))

res13: Array[(Double, Double)] = Array(
  (1.0, 1.0119528619721367),
  (1.0, 0.7850917056962649),
  (0.2, 0.7919950327136843),
  (1.0, 0.7366057622779553),
  (0.2, 0.7503403590685356),
  (1.0, 0.918508425086659),
  (1.0, 0.7970787108554558),
  (1.0, 0.8389514107260572),
  (0.2, 0.827966876830238),
  (0.6, 0.7579016201423178)
)

## Decision Trees Regression
---

### Train

In [15]:
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.feature.VectorIndexer
import org.apache.spark.ml.regression.DecisionTreeRegressionModel
import org.apache.spark.ml.regression.DecisionTreeRegressor

val featureIndexer = new VectorIndexer()
  .setInputCol("features")
  .setOutputCol("indexedFeatures")
  .setMaxCategories(4)
  .fit(trainingDF)

val dt = new DecisionTreeRegressor()
  .setLabelCol("label")
  .setFeaturesCol("indexedFeatures")

val pipeline = new Pipeline()
  .setStages(Array(featureIndexer, dt))

val dtModel = pipeline.fit(trainingDF)




head at MetadataUtils.scala:51

1 / 1

treeReduce at VectorIndexer.scala:150

16 / 16

treeReduce at VectorIndexer.scala:150

...

mapPartitions at RandomForest.scala:644

16 / 16

collectAsMap at RandomForest.scala:663

16 / 16

import org.apache.spark.ml.Pipeline

import org.apache.spark.ml.evaluation.RegressionEvaluator

import org.apache.spark.ml.feature.VectorIndexer

import org.apache.spark.ml.regression.DecisionTreeRegressionModel

import org.apache.spark.ml.regression.DecisionTreeRegressor


featureIndexer: org.apache.spark.ml.feature.VectorIndexerModel = VectorIndexerModel: uid=vecIdx_717a64adf82d, numFeatures=128, handleInvalid=error
dt: DecisionTreeRegressor = dtr_4e2f7af64ad8
pipeline: Pipeline = pipeline_bb25bd808c3f
dtModel: org.apache.spark.ml.PipelineModel = pipeline_bb25bd808c3f

### Evaluate

In [20]:
val dtPredictions = dtModel.transform(testingDF)
// Select (prediction, true label) and compute test error.
val evaluator = new RegressionEvaluator()
  .setLabelCol("label")
  .setPredictionCol("prediction")
  .setMetricName("rmse")
val rmse = evaluator.evaluate(dtPredictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

val treeModel = dtModel.stages(1).asInstanceOf[DecisionTreeRegressionModel]
println(s"Learned regression tree model:\n ${treeModel.toDebugString}")

treeAggregate at Statistics.scala:58

16 / 16

treeAggregate at Statistics.scala:58

4 / 4

Root Mean Squared Error (RMSE) on test data = 0.21329785760436926
Learned regression tree model:
 DecisionTreeRegressionModel: uid=dtr_4e2f7af64ad8, depth=5, numNodes=63, numFeatures=128
  If (feature 30 <= -0.5243920721612841)
   If (feature 57 <= -1.5888873824480543)
    If (feature 113 <= 0.6768750264365906)
     If (feature 37 <= 3.1649567515321735)
      If (feature 97 <= -0.781569487971845)
       Predict: 0.9704222222222223
      Else (feature 97 > -0.781569487971845)
       Predict: 0.9361872444096402
     Else (feature 37 > 3.1649567515321735)
      If (feature 17 <= -0.6818675222968513)
       Predict: 0.6064516129032259
      Else (feature 17 > -0.6818675222968513)
       Predict: 0.9039999999999999
    Else (feature 113 > 0.6768750264365906)
     If (feature 78 <= -1.3557317597566716)
      If (feature 127 <= 1.27580543930622)
       Predict: 0.9838220424671382
      Else (feature 127 > 1.27580543930622)
       Predict: 0.8000000000000004
     Else (feature 78 > -1.35573175

dtPredictions: DataFrame = [label: double, features: vector ... 2 more fields]
evaluator: RegressionEvaluator = RegressionEvaluator: uid=regEval_9b354b31242f, metricName=rmse, throughOrigin=false
rmse: Double = 0.21329785760436926
treeModel: DecisionTreeRegressionModel = DecisionTreeRegressionModel: uid=dtr_4e2f7af64ad8, depth=5, numNodes=63, numFeatures=128

### Predict

In [21]:
// Make predictions.
dtPredictions.show()
// sample.map(p=>(p.getDouble(0),model.predict(fromML(p.getAs[Vector](1)))))

show at cmd20.sc:1

1 / 1

+-----+--------------------+--------------------+------------------+
|label|            features|     indexedFeatures|        prediction|
+-----+--------------------+--------------------+------------------+
|  0.2|[-0.9000950601860...|[-0.9000950601860...|0.7062567681386114|
|  1.0|[-0.9322016323397...|[-0.9322016323397...| 0.802393935363745|
|  0.8|[-0.3934224748757...|[-0.3934224748757...| 0.802393935363745|
|  0.2|[-1.0176151816014...|[-1.0176151816014...|0.8617486689332223|
|  1.0|[-0.9302095340987...|[-0.9302095340987...|0.8279875610839625|
|  1.0|[0.09125376997453...|[0.09125376997453...|0.8865544381285532|
|  0.8|[-1.0086159592524...|[-1.0086159592524...| 0.802393935363745|
|  0.2|[-1.1424247837867...|[-1.1424247837867...|0.5239849624060149|
|  0.8|[-0.5844029567671...|[-0.5844029567671...|0.6674826689774697|
|  0.2|[0.17110813899282...|[0.17110813899282...|0.7800522061300098|
|  0.4|[-1.1433561208052...|[-1.1433561208052...|0.5239849624060149|
|  0.6|[-0.6022630619947...|[-0.60

## K-Means Clustering
---

### Train

In [22]:
import org.apache.spark.ml.clustering.KMeans
import org.apache.spark.ml.evaluation.ClusteringEvaluator

// Trains a k-means model.
val kmeans = new KMeans().setK(10).setSeed(1L)
val kModel = kmeans.fit(trainingDF)


// Save and load model
//clusters.save(sc, "target/org/apache/spark/KMeansExample/KMeansModel")
//val sameModel = KMeansModel.load(sc, "target/org/apache/spark/KMeansExample/KMeansModel")


takeSample at KMeans.scala:377

16 / 16

takeSample at KMeans.scala:377

16 / 16

sum at KMeans.scala:397

...

collect at ClusteringSummary.scala:49

16 / 16

collect at ClusteringSummary.scala:49

200 / 200

import org.apache.spark.ml.clustering.KMeans

import org.apache.spark.ml.evaluation.ClusteringEvaluator

// Trains a k-means model.

kmeans: KMeans = kmeans_23e3a0c6019b
kModel: org.apache.spark.ml.clustering.KMeansModel = KMeansModel: uid=kmeans_23e3a0c6019b, k=5, distanceMeasure=euclidean, numFeatures=128

### Evaluate

In [24]:
// Make predictions
val kPredictions = kModel.transform(testingDF)

// Evaluate clustering by computing Silhouette score
val evaluator = new ClusteringEvaluator()

val silhouette = evaluator.evaluate(kPredictions)
println(s"Silhouette with squared euclidean distance = $silhouette")

// Shows the result.
println("Cluster Centers: ")
kModel.clusterCenters.foreach(println)


head at MetadataUtils.scala:51

1 / 1

map at ClusteringMetrics.scala:300

16 / 16

collectAsMap at ClusteringMetrics.scala:321

...

16 / 16

collect at ClusteringMetrics.scala:102

1 / 1

Silhouette with squared euclidean distance = 0.6344750381055456
Cluster Centers: 
[-0.7453775749528371,0.3465392137301101,-0.7544586493361038,0.08563423053180477,-0.3810314122839774,-0.1382209920984032,-0.19240053550298467,-0.1476535027012662,-0.07535450029428199,-0.05358207071909258,0.34871321061429017,0.2756207504100873,0.05778603956362022,-0.1402824172343614,-0.2573316151319816,0.3737449048654583,0.10820341605440932,1.1439756151828744,0.8741280711090786,-0.021653826296977263,0.6141028634255391,-0.17415902602725788,-0.08612805521790412,-0.14808893191381822,-0.3137375040758514,-0.43291165136655363,-0.32495670159439344,1.2210258231692739,-0.3360711258432141,0.06767436760853941,-0.29641174051677416,0.06070941069471358,0.6116964198445572,-0.3893119035892756,-1.1571206080523806,0.1730622550708945,0.03280258298677624,-0.5241830893342427,0.5089477149034923,-0.01943002805722149,0.5983670181895282,0.4658665504016947,-0.38541165333281385,0.8341649448184704,-0.08845814135908964,-0.1980849922761

kPredictions: DataFrame = [label: double, features: vector ... 1 more field]
evaluator: ClusteringEvaluator = ClusteringEvaluator: uid=cluEval_1b0fc0ac95a9, metricName=silhouette, distanceMeasure=squaredEuclidean
silhouette: Double = 0.6344750381055456

### Predict

In [25]:
kPredictions.show()

show at cmd24.sc:1

1 / 1

+-----+--------------------+----------+
|label|            features|prediction|
+-----+--------------------+----------+
|  0.2|[-0.9000950601860...|         0|
|  1.0|[-0.9322016323397...|         0|
|  0.8|[-0.3934224748757...|         0|
|  0.2|[-1.0176151816014...|         0|
|  1.0|[-0.9302095340987...|         0|
|  1.0|[0.09125376997453...|         0|
|  0.8|[-1.0086159592524...|         0|
|  0.2|[-1.1424247837867...|         0|
|  0.8|[-0.5844029567671...|         0|
|  0.2|[0.17110813899282...|         0|
|  0.4|[-1.1433561208052...|         0|
|  0.6|[-0.6022630619947...|         0|
|  0.8|[-1.1259042717062...|         0|
|  0.2|[-0.2533524093331...|         0|
|  0.2|[-0.9495015236977...|         0|
|  0.8|[-0.9013505560531...|         0|
|  0.6|[-1.0881728264052...|         0|
|  1.0|[0.42418014201439...|         0|
|  1.0|[-0.6358369209998...|         2|
|  0.8|[0.25375451890388...|         0|
+-----+--------------------+----------+
only showing top 20 rows

